## Before running

A virtual environment can be created using 
- 'pipenv install'
- 'pipenv shell'

This will allow us to all use the same packages and versions. They are listed in the Pipfile

In [1]:
from refactoring import *
# from ase.geometry.analysis import Analysis
# import pandas as pd
# from sklearn.model_selection import RepeatedKFold
# from dataclasses import dataclass
# from quippy import descriptors
# import ase
# import subprocess
# from pathlib import Path
# import os
# import pickle as pkl
# from random import sample, shuffle, choice, choices
# import numpy as np


## Inputs

Dictionaries are taken as input from a parameter file, they contain the parameters for each soap descriptor

In [2]:
descDict1 = {'lower': 1, 'upper': 50, 'centres': '{8, 7, 6, 1, 16, 17, 9}',
             'neighbours': '{8, 7, 6, 1, 16, 17, 9}', 'mu': 0, 
             'mu_hat': 0, 'nu': 2, 'nu_hat': 0, 'mutation_chance': 0.50, 
             'min_cutoff': 1, 'max_cutoff': 50, 'min_sigma': 0.1, 
             'max_sigma': 0.9,
             'message_steps': 8}

descDict2 = {'lower': 51, 'upper': 100, 'centres': '{8, 7, 6, 1, 16, 17, 9}',
             'neighbours': '{8, 7, 6, 1, 16, 17, 9}', 'mu': 0, 
             'mu_hat': 0, 'nu': 2, 'nu_hat': 0, 'mutation_chance': 0.50,
             'min_cutoff': 51, 'max_cutoff': 100, 'min_sigma': 1.1, 
             'max_sigma': 1.9,
             'message_steps': 8}

Other parameters are also taken as input. These are automatically checked that the parameters are viable

In [3]:
num_gens = 100
best_sample, lucky_few, population_size, number_of_children = 4, 2, 12, 4
early_stop = 2
early_number = 3 
min_generations = 5

## GeneParameter

GeneParameter class is created from each descriptor dictionary. 

In [4]:
params1 = GeneParameters(**descDict1)
params2 = GeneParameters(**descDict2)

In [5]:
params1

GeneParameters(lower=1, upper=50, centres='{8, 7, 6, 1, 16, 17, 9}', neighbours='{8, 7, 6, 1, 16, 17, 9}', mu=0, mu_hat=0, nu=2, nu_hat=0, mutation_chance=0.5, min_cutoff=1, max_cutoff=50, min_sigma=0.1, max_sigma=0.9, message_steps=8)

## GeneSet

We can use these classes to create a specific set of parameters that are consistant with these values. This returns a randomly generated GeneSet class

In [6]:
example_gene_set = params1.make_gene_set()
example_gene_set

GeneSet(9, 4, 13, 0.83)

We can get the parameters used to create the GeneSet class

In [7]:
example_gene_set.gene_parameters

GeneParameters(lower=1, upper=50, centres='{8, 7, 6, 1, 16, 17, 9}', neighbours='{8, 7, 6, 1, 16, 17, 9}', mu=0, mu_hat=0, nu=2, nu_hat=0, mutation_chance=0.5, min_cutoff=1, max_cutoff=50, min_sigma=0.1, max_sigma=0.9, message_steps=8)

We can get a descriptor string to be used as an input for getting SOAPs

In [8]:
example_gene_set.get_soap_string()

'soap cutoff=9 l_max=4 n_max=13 atom_sigma=0.83 n_Z=7 Z={8, 7, 6, 1, 16, 17, 9} n_species=7 species_Z={8, 7, 6, 1, 16, 17, 9} mu=0 mu_hat=0 nu=2 nu_hat=0'

We can also mutate the gene using the mutation chance in the GeneParameters class

In [9]:
print(f"Before mutation {example_gene_set}")
example_gene_set.mutate_gene()
print(f"After mutation {example_gene_set}")

Before mutation [9, 4, 13, 0.83]
After mutation [9, 4, 20, 0.81]


## Individual

An Individual is made up of a list of GeneSet classes.

In [10]:
example_gene_set_two = params2.make_gene_set()
gene_set_list = [example_gene_set, example_gene_set_two]
example_individual = Individual(gene_set_list)
example_individual

Individual(['GeneSet(9, 4, 20, 0.81)', 'GeneSet(92, 88, 84, 1.8)'])

Getting the score for an indivudual

In [11]:
example_individual.get_score()
example_individual.score

101

Breeding two individuals to create a child. Mutation is automatically performed during this

In [12]:
example_individual_two = Individual(gene_set_list)
print(f"Breeding {example_individual} with {example_individual_two}")
child = breed_individuals(example_individual, example_individual_two)
print(f"Created child {child}")

Breeding Individual(['[9, 4, 20, 0.81]', '[92, 88, 84, 1.8]']) with Individual(['[9, 4, 20, 0.81]', '[92, 88, 84, 1.8]'])
Created child Individual(['[9, 4, 20, 0.81]', '[75, 51, 69, 1.16]'])


## Population

A Population is a collection of Individual classes. This can be created using a list of GeneParameter classes

In [13]:
gene_parameters = [params1, params2]
pop = Population(best_sample, lucky_few, population_size, 
                 number_of_children, gene_parameters, 
                 maximise_scores = True)
pop

Population(4, 2, 12, 4, [GeneParameters(lower=1, upper=50, centres='{8, 7, 6, 1, 16, 17, 9}', neighbours='{8, 7, 6, 1, 16, 17, 9}', mu=0, mu_hat=0, nu=2, nu_hat=0, mutation_chance=0.5, min_cutoff=1, max_cutoff=50, min_sigma=0.1, max_sigma=0.9, message_steps=8), GeneParameters(lower=51, upper=100, centres='{8, 7, 6, 1, 16, 17, 9}', neighbours='{8, 7, 6, 1, 16, 17, 9}', mu=0, mu_hat=0, nu=2, nu_hat=0, mutation_chance=0.5, min_cutoff=51, max_cutoff=100, min_sigma=1.1, max_sigma=1.9, message_steps=8)], True)

To initialise the population

In [14]:
pop.initialise_population()

Initial population of size 12 generated


If you want a way of neatly seeing what individuals are in the population

In [15]:
pop.print_population()

Individual(['[35, 40, 8, 0.63]', '[88, 58, 68, 1.21]']) has a score of: 123
Individual(['[15, 42, 25, 0.61]', '[66, 57, 93, 1.57]']) has a score of: 81
Individual(['[12, 19, 32, 0.37]', '[62, 92, 96, 1.51]']) has a score of: 74
Individual(['[16, 4, 28, 0.67]', '[87, 54, 92, 1.71]']) has a score of: 103
Individual(['[22, 17, 4, 0.49]', '[86, 95, 90, 1.46]']) has a score of: 108
Individual(['[42, 18, 27, 0.85]', '[72, 60, 80, 1.65]']) has a score of: 114
Individual(['[16, 3, 32, 0.63]', '[98, 71, 82, 1.14]']) has a score of: 114
Individual(['[36, 7, 3, 0.88]', '[88, 91, 82, 1.36]']) has a score of: 124
Individual(['[46, 8, 16, 0.41]', '[97, 52, 65, 1.53]']) has a score of: 143
Individual(['[39, 15, 21, 0.26]', '[72, 70, 66, 1.4]']) has a score of: 111
Individual(['[38, 9, 5, 0.21]', '[86, 55, 85, 1.8]']) has a score of: 124
Individual(['[12, 25, 34, 0.51]', '[87, 58, 87, 1.1]']) has a score of: 99


The next generation can then be generated 

In [16]:
pop.next_generation()
pop.print_population()

Individual(['[12, 25, 29, 0.32]', '[98, 71, 82, 1.28]']) has a score of: 110
Individual(['[9, 37, 6, 0.46]', '[86, 58, 68, 1.21]']) has a score of: 95
Individual(['[30, 7, 16, 0.36]', '[97, 52, 70, 1.5]']) has a score of: 127
Individual(['[35, 9, 3, 0.81]', '[86, 90, 85, 1.21]']) has a score of: 121
Individual(['[3, 38, 34, 0.63]', '[55, 64, 61, 1.14]']) has a score of: 58
Individual(['[39, 40, 26, 0.85]', '[95, 95, 68, 1.21]']) has a score of: 134
Individual(['[3, 32, 34, 0.51]', '[89, 71, 87, 1.54]']) has a score of: 92
Individual(['[46, 47, 16, 0.49]', '[97, 61, 92, 1.36]']) has a score of: 143
Individual(['[4, 5, 32, 0.78]', '[87, 98, 82, 1.14]']) has a score of: 91
Individual(['[3, 5, 3, 0.41]', '[56, 52, 94, 1.18]']) has a score of: 59
Individual(['[45, 27, 16, 0.41]', '[81, 64, 51, 1.5]']) has a score of: 126
Individual(['[45, 35, 42, 0.7]', '[53, 90, 85, 1.63]']) has a score of: 98


So to run the full GA 

In [17]:
for _ in range(num_gens):
    pop.next_generation()
pop.print_population()

Individual(['[26, 46, 40, 0.35]', '[90, 70, 96, 1.68]']) has a score of: 116
Individual(['[43, 49, 35, 0.56]', '[84, 99, 65, 1.56]']) has a score of: 127
Individual(['[27, 43, 37, 0.84]', '[93, 97, 96, 1.69]']) has a score of: 120
Individual(['[9, 27, 40, 0.22]', '[90, 68, 63, 1.68]']) has a score of: 99
Individual(['[7, 44, 18, 0.24]', '[55, 96, 78, 1.67]']) has a score of: 62
Individual(['[34, 20, 18, 0.48]', '[99, 86, 55, 1.75]']) has a score of: 133
Individual(['[46, 24, 18, 0.23]', '[97, 51, 65, 1.33]']) has a score of: 143
Individual(['[38, 19, 45, 0.26]', '[84, 59, 90, 1.41]']) has a score of: 122
Individual(['[9, 13, 18, 0.16]', '[64, 92, 78, 1.79]']) has a score of: 73
Individual(['[4, 19, 30, 0.12]', '[85, 68, 90, 1.56]']) has a score of: 89
Individual(['[48, 22, 11, 0.66]', '[55, 94, 66, 1.54]']) has a score of: 103
Individual(['[49, 27, 40, 0.84]', '[93, 99, 96, 1.68]']) has a score of: 142


## BestHistory

BestHistory is a class to store the history and check convergence criteria. So the entire GA can be run, printed, and saved using the following code snippet:

In [18]:
hist = BestHistory(early_stop, early_number, min_generations)
pop = Population(best_sample, lucky_few, population_size, 
                 number_of_children, gene_parameters, 
                 maximise_scores = True)

pop.initialise_population()    
for gen in range(num_gens):
    if hist.converged:
        break
    print(f"Generation {gen}")
    pop.next_generation()
    hist.append(pop)
    print("-------")

Initial population of size 12 generated
Generation 0
Best Individual Individual(['[45, 5, 9, 0.17]', '[99, 89, 51, 1.37]']) with a score of 144 added to history
-------
Generation 1
Best Individual Individual(['[47, 47, 4, 0.12]', '[98, 98, 69, 1.54]']) with a score of 145 added to history
-------
Generation 2
Best Individual Individual(['[47, 5, 34, 0.12]', '[99, 60, 57, 1.31]']) with a score of 146 added to history
-------
Generation 3
Best Individual Individual(['[47, 23, 34, 0.37]', '[99, 81, 93, 1.39]']) with a score of 146 added to history
-------
Generation 4
Best Individual Individual(['[47, 1, 4, 0.29]', '[98, 81, 96, 1.71]']) with a score of 145 added to history
SOAP_GAS has converged
-------


There now exists the entire history of the best Individuals throughout each generation that can be saved and easily accessed. 

In [19]:
vars(hist)

{'history': [Individual(['GeneSet(45, 5, 9, 0.17)', 'GeneSet(99, 89, 51, 1.37)']),
  Individual(['GeneSet(47, 47, 4, 0.12)', 'GeneSet(98, 98, 69, 1.54)']),
  Individual(['GeneSet(47, 5, 34, 0.12)', 'GeneSet(99, 60, 57, 1.31)']),
  Individual(['GeneSet(47, 23, 34, 0.37)', 'GeneSet(99, 81, 93, 1.39)']),
  Individual(['GeneSet(47, 1, 4, 0.29)', 'GeneSet(98, 81, 96, 1.71)'])],
 'converged': True,
 'early_stop': 2,
 'early_number': 3,
 'min_generations': 5}

In [20]:
example_gene_set.cutoff = 5
example_gene_set.l_max = 3
example_gene_set.n_max = 6

In [21]:
test_ind = Individual([example_gene_set])
conf_s, data = get_conf()

conf_s file exists


In [22]:
conf_s

[Atoms(symbols='COC6NCOC4ClC7O2C2O2H18', pbc=False),
 Atoms(symbols='C2ONC6OH9', pbc=False),
 Atoms(symbols='C3SO2C6FCONC7NCF3OH14', pbc=False),
 Atoms(symbols='C6NC6NC3OC7NOH25', pbc=False),
 Atoms(symbols='C25H34O6', pbc=False),
 Atoms(symbols='C3SCONC5O2H15', pbc=False),
 Atoms(symbols='COC6OC2NC3OC12NOH26', pbc=False),
 Atoms(symbols='C10N2C6SO2NCF3H14', pbc=False),
 Atoms(symbols='C9ONCOCCl2ONO2H12', pbc=False),
 Atoms(symbols='C2NC2NC22H28', pbc=False),
 Atoms(symbols='C14ClOC6NCH26', pbc=False),
 Atoms(symbols='C19ClNC2NCH17', pbc=False),
 Atoms(symbols='C12OC10NOH27', pbc=False),
 Atoms(symbols='C2OCSCONC3NCONFH10', pbc=False),
 Atoms(symbols='C10OC8OH24', pbc=False),
 Atoms(symbols='C9ONC6FC9FO2H21', pbc=False),
 Atoms(symbols='C2OCOC2NC9Cl2CO2C3H19', pbc=False),
 Atoms(symbols='C14FCO2H13', pbc=False),
 Atoms(symbols='C7O2C3O2NC7NC2ClH17', pbc=False),
 Atoms(symbols='C23H28ClN3O5S', pbc=False),
 Atoms(symbols='CNC6SO2NSO2NClH8', pbc=False),
 Atoms(symbols='C5OC14OCO2COH30', p

In [23]:
test_ind.comp_soaps(conf_s, data)

Getting soap for Atoms(symbols='COC6NCOC4ClC7O2C2O2H18', pbc=False)
soap cutoff=5 l_max=3 n_max=6 atom_sigma=0.81 n_Z=7 Z={8, 7, 6, 1, 16, 17, 9} n_species=7 species_Z={8, 7, 6, 1, 16, 17, 9} mu=0 mu_hat=0 nu=2 nu_hat=0
(47, 3613)
(47, 47)
Getting soap for Atoms(symbols='C2ONC6OH9', pbc=False)
soap cutoff=5 l_max=3 n_max=6 atom_sigma=0.81 n_Z=7 Z={8, 7, 6, 1, 16, 17, 9} n_species=7 species_Z={8, 7, 6, 1, 16, 17, 9} mu=0 mu_hat=0 nu=2 nu_hat=0
(20, 3613)
(20, 20)
Getting soap for Atoms(symbols='C3SO2C6FCONC7NCF3OH14', pbc=False)
soap cutoff=5 l_max=3 n_max=6 atom_sigma=0.81 n_Z=7 Z={8, 7, 6, 1, 16, 17, 9} n_species=7 species_Z={8, 7, 6, 1, 16, 17, 9} mu=0 mu_hat=0 nu=2 nu_hat=0
(43, 3613)
(43, 43)
Getting soap for Atoms(symbols='C6NC6NC3OC7NOH25', pbc=False)
soap cutoff=5 l_max=3 n_max=6 atom_sigma=0.81 n_Z=7 Z={8, 7, 6, 1, 16, 17, 9} n_species=7 species_Z={8, 7, 6, 1, 16, 17, 9} mu=0 mu_hat=0 nu=2 nu_hat=0
(52, 3613)
(52, 52)
Getting soap for Atoms(symbols='C25H34O6', pbc=False)
soap c

(63, 3613)
(63, 63)
Getting soap for Atoms(symbols='C25H38O5', pbc=False)
soap cutoff=5 l_max=3 n_max=6 atom_sigma=0.81 n_Z=7 Z={8, 7, 6, 1, 16, 17, 9} n_species=7 species_Z={8, 7, 6, 1, 16, 17, 9} mu=0 mu_hat=0 nu=2 nu_hat=0
(68, 3613)
(68, 68)
Getting soap for Atoms(symbols='C24H32O4S', pbc=False)
soap cutoff=5 l_max=3 n_max=6 atom_sigma=0.81 n_Z=7 Z={8, 7, 6, 1, 16, 17, 9} n_species=7 species_Z={8, 7, 6, 1, 16, 17, 9} mu=0 mu_hat=0 nu=2 nu_hat=0
(61, 3613)
(61, 61)
Getting soap for Atoms(symbols='C13SOC5FC2O2H17', pbc=False)
soap cutoff=5 l_max=3 n_max=6 atom_sigma=0.81 n_Z=7 Z={8, 7, 6, 1, 16, 17, 9} n_species=7 species_Z={8, 7, 6, 1, 16, 17, 9} mu=0 mu_hat=0 nu=2 nu_hat=0
(42, 3613)
(42, 42)
Getting soap for Atoms(symbols='C16OC2NC8H29', pbc=False)
soap cutoff=5 l_max=3 n_max=6 atom_sigma=0.81 n_Z=7 Z={8, 7, 6, 1, 16, 17, 9} n_species=7 species_Z={8, 7, 6, 1, 16, 17, 9} mu=0 mu_hat=0 nu=2 nu_hat=0
(57, 3613)
(57, 57)
Getting soap for Atoms(symbols='C3NCNC2NCNCNOCPO3H14', pbc=False

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 32 is different from 33)

In [24]:
test_ind.soaps.shape

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers, optimizers, Model, backend
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.callbacks import EarlyStopping
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
def cv_split(individual, splits, repeats, random_state):
    """
    Returns split indices for train and test sets
    """
    cv = RepeatedKFold(n_splits = splits, n_repeats = repeats, random_state = random_state)
    for train_index, test_index in cv.split(soaps):
        print(train_index, test_index) 

In [ ]:
def scaleData(train, test):
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train)
    test_scaled = scaler.transform(test)

    return train_scaled, test_scaled, scaler

In [ ]:
def buildModel(X):
    backend.clear_session()
    input_layer = Input(X.shape[1])
    hidden_layer = input_layer
    for layer in [30,30,30]:
        hidden_layer = Dense(layer, activation='relu')(hidden_layer)

    output_layer = Dense(units=1, activation='linear')(hidden_layer)
    model = Model(input_layer, output_layer)
    model.compile(loss='mean_squared_error', optimizer=optimizers.Adam(learning_rate=0.01), metrics=['mean_squared_error'])

    return model

In [ ]:

def get_scores(Individual, train_index, test_index, scaling = None, **kwargs):
    estimator = buildModel(Individual.soaps)
    X_train, X_test, X_scaler = scaleData(Individual.soaps[train_index], Individual.soaps[test_index])
    y_train, y_test, y_scaler = scaleData(Individual.targets[train_index].reshape(-1,1), Individual.targets[test_index].reshape(-1,1))
    scores = scorerNN(estimator, X_train, X_test, y_train, y_test, y_scaler)
    print(scores[0])
    return scores

In [ ]:
def scorerNN(estimator, X_train, X_test, y_train, y_test, y_scaler):
    """ Scoring function for use with NN regressor. Added by Matt. """

    callback = EarlyStopping(monitor='val_loss', patience=50)
    estimator.fit(X_train, y_train, callbacks=[callback], validation_split=0.1, epochs=200, verbose=False)
    y_test_pred, y_train_pred = estimator.predict(X_test, verbose=False), estimator.predict(X_train, verbose=False)
    y_test_pred, y_train_pred = y_scaler.inverse_transform(y_test_pred), y_scaler.inverse_transform(y_train_pred)
    y_test = np.ravel(y_test)
    y_train = np.ravel(y_train)
    testCorr = pearsonr(y_test, y_test_pred)[0]
    trainCorr = pearsonr(y_train, y_train_pred)[0]
    testMSE = mean_squared_error(y_test, y_test_pred)
    trainMSE =  mean_squared_error(y_train, y_train_pred)
    return (2 * (trainMSE * (1-trainCorr)) + (testMSE * (1-testCorr))), X_train, X_test, y_train, y_test, y_test_pred, y_train_pred

In [ ]:
cv = RepeatedKFold(n_splits = 5, n_repeats = 2, random_state = 6)
for train_index, test_index in cv.split(test_ind.soaps):
    get_scores(test_ind, train_index, test_index)

In [ ]:
type(t.dtype)

In [ ]:
def build_model(individual):
    if individual.targets.dtype == float:
        target_type = 'regression'
    elif individual.targets.dtype == (int or str):
        target_type = 'classification'
    estimator = 3
    return estimator, target_type

In [ ]:
build_model(test_ind)